In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

def energia_diaria(archivo_json):
   
    # Cargar el "DataFrame"
    df = pd.read_json(archivo_json) 

    # Convertir en un array de NumPy
    datos = np.array(df)  

    # Crear vector con todos los valores horarios de demanda
    demanda = []

    # Extraer la magnitud de la demanda para todas las horas
    for hora in range(len(datos)):
        demanda.append(datos[hora][0]['MW'])

    # Separar las magnitudes en grupos de 24 (24 h)
    demanda = np.split(np.array(demanda), len(demanda) / 24)

    # Crear vector para almacenar la energía a partir de la demanda
    energia = []

    # Calcular la energía diaria por la regla del trapecio
    for dia in range(len(demanda)):
        E = round(np.trapz(demanda[dia]), 2)
        energia.append(E)

    return energia 


In [2]:
import numpy as np

def definicion_estados(vector_energia, estados):
 
    minimo = np.min(vector_energia)
    maximo = np.max(vector_energia)
    segmento = (maximo - minimo)/estados
    vector_estados = np.empty(len(vector_energia))
    
    for i, dia in enumerate(vector_energia):
        diferencia = dia - minimo
        proporcion = diferencia // segmento
        vector_estados[i] = proporcion + 1
        
    return vector_estados

In [9]:
import numpy as np

def probabilidad_transicion(vector_estados, numero_estados, presente, futuro):

    
    # Recorrer el vector_estados
    ocurrencias_i = 0
    ocurrencias_i_j = 0
    for i, estado in enumerate(vector_estados[0:-1]):
        if estado == presente:
            ocurrencias_i += 1
            if vector_estados[i+1] == futuro:
                ocurrencias_i_j += 1
    
    # Cálculo de la probabilidad
    probabilidad = ocurrencias_i_j / ocurrencias_i
    
    return probabilidad

In [12]:
# import matplotlib.pyplot as plt

# Importar los datos y calcular la energía diaria
vector_energia = energia_diaria('demanda_2019.json')

# Definir los estados
numero_estados = 10
vector_estados = definicion_estados(vector_energia, numero_estados)
#print(vector_estados)

states = [1,2,3,4,5,6,7,8,9,10]

trans = np.zeros((len(states), len(states)))

filas = 1
columnas = 1

for i, filas in enumerate(states):
    for j, columnas in enumerate(states):
        Pi_J = probabilidad_transicion(vector_estados, numero_estados, filas, columnas)
        trans[i,j] = Pi_J
        
print (np.round(trans,4))

# Graficar la evolución de los estados
#plt.plot(vector_estados)
#plt.xlabel('Día del año')
#plt.ylabel('Consumo de energía (estado)')
#plt.show()

# Definir la probabilidad de transición de "i" a "j"
i, j = 5, 9
t = 2
Pi_ij = probabilidad_transicion(vector_estados, numero_estados, i, j) * t
print('Pi_ij =', Pi_ij)

[[0.     0.     0.     0.     0.     1.     0.     0.     0.     0.    ]
 [0.     0.     0.     0.5    0.     0.5    0.     0.     0.     0.    ]
 [0.     0.     0.     0.     0.     0.     0.7143 0.2857 0.     0.    ]
 [0.     0.0526 0.0526 0.     0.     0.     0.2105 0.5263 0.1579 0.    ]
 [0.     0.     0.     0.0909 0.     0.     0.0909 0.1818 0.5455 0.0909]
 [0.     0.     0.2941 0.4118 0.     0.0588 0.0588 0.     0.     0.1765]
 [0.     0.0333 0.0333 0.3    0.2    0.0333 0.     0.3667 0.     0.0333]
 [0.     0.     0.     0.0135 0.0676 0.1351 0.1081 0.473  0.1892 0.0135]
 [0.     0.     0.     0.     0.     0.0545 0.1818 0.1818 0.4    0.1818]
 [0.     0.     0.     0.     0.     0.     0.027  0.1351 0.2432 0.5676]]
Pi_ij = 1.0909090909090908
